In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

### Read in Data Set

In [3]:
raw_data = pd.read_csv('household_power_consumption.txt', sep=';', na_values='?', index_col=0,
                       parse_dates=[[0,1]], infer_datetime_format=True, dayfirst=True)

In [4]:
for i in range(raw_data.shape[1]):
    bad_cnt = raw_data.iloc[:, i].isnull().sum()
    print('Column %d has %d bad values.' % (i, bad_cnt))

Column 0 has 25979 bad values.
Column 1 has 25979 bad values.
Column 2 has 25979 bad values.
Column 3 has 25979 bad values.
Column 4 has 25979 bad values.
Column 5 has 25979 bad values.
Column 6 has 25979 bad values.


In [285]:
print(raw_data.shape)
# print(raw_data.iloc[4,[2,3,6,7,8]])
# print(raw_data.head)

(2075259, 7)


### Fill missing data with average curve
This method of filling the data will replace each missing value with the average of the last good value and the next good value. If there are multiple missing values, the last good value will be updated as each value is filled in. For example, if the series of data is [0, ?, ?, 4], the filled in set will be [0, 2, 3, 4].  
I think this is a reasonable way to fill in power data since power is continuous and can't change value instantaneously. It might be problematic for the regions where there are many values (multiple days worth) missing, though.

In [7]:
filled_data = raw_data.copy()

for col in range(1):#filled_data.shape[1]):
    last_row = -2
    
    null_series = filled_data.iloc[:, col].isnull()
    null_indices = np.where(null_series.values == True)[0].tolist()
    print(type(null_indices))
    skip_cnt = 0
    for i in range(len(null_indices)):
        row = null_indices[i]
        
        if (last_row == row - 1):
            new_val = (last_good + next_good) / 2
            filled_data.iloc[row] = new_val
            last_good = new_val
            continue
            
        if (i == len(null_indices) - 1):
            # This is the last null value in the column. The next row is good.
            filled_data.iloc[row] = (filled_data.iloc[row - 1] + filled_data.iloc[row + 1]) / 2
        else:
            good_idx = 0
            for k in range(i, len(null_indices)):
                if (null_indices[k] != null_indices[k + 1] - 1):
                    good_idx = k
                    break
                elif (k == len(null_indices) - 1):
                    good_idx = k
            next_good = filled_data.iloc[null_indices[good_idx] + 1]
            last_good = filled_data.iloc[row - 1]
            new_val = (last_good + next_good) / 2
            filled_data.iloc[row] = new_val
            last_good = new_val
            last_row = row
                
    print('Column %d cleaned' % col)

<class 'list'>
Column 0 cleaned


### Add a new column for apparent power
If I remember my power systems courses correctly, the power utility needs to consider apparent power, which is a combination of both active and reactive power. This next bit of code will calculate the apparent power and add it as a new column. This column will be useful if we want to do analysis related to how much power the utility needs to generate.

In [9]:
# Add apparent power column
filled_data['Global_apparent_power'] = np.sqrt(filled_data['Global_active_power']**2 +
                                              filled_data['Global_reactive_power']**2)

### Aggregate data by hour/day/season rather than by minute